# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
path = r'/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data/orders_wrangled.csv'
print (path)
df_ords=pd.read_csv(path, index_col=False)
path2=r'/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Original Data/products.csv'
print (path2)
df_prods=pd.read_csv(path2, index_col=False)
path3=r'/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Original Data/departments 2.csv'
print(path3)
df_dep=pd.read_csv(path3, index_col=False)
path5=r'/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data/orders_products_combined.pkl'
print(path5)
df_merged_large = pd.read_pickle(path5)

/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data/orders_wrangled.csv


FileNotFoundError: [Errno 2] No such file or directory: '/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data/orders_wrangled.csv'

# Mix Data Sample

In [67]:
#create a data frame and create a mixed type column
df_test=pd.DataFrame()
df_test['mix']=['a','b',1,True]

In [68]:
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [69]:
#check for mix types
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [70]:
df_test['mix'] = df_test['mix'].astype('str')

In [71]:
#check for mix values of Products
for col in df_prods.columns.tolist(): 
  weird = (df_prods[[col]].applymap(type) != df_prods[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_prods[weird]) > 0:
    print (col, 'mixed')
  else:
    print(col, 'consistent')

product_id consistent
product_name mixed
aisle_id consistent
department_id consistent
prices consistent


The only columns with mixed values is the product_name 

In [72]:
#check for mix values of orders
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col, 'mixed')
  else:
    print(col,'consistent')

Unnamed: 0 consistent
order_id consistent
user_id consistent
order_number consistent
order_day_of_week consistent
order_hour_of_day consistent
days_since_last_order consistent


# Missing Values

In [73]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [74]:
#View of the 16 missing data values
df_prods_nan=df_prods[df_prods['product_name'].isnull()==True]
df_prods_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


Code to input the mean or median for missing values
df['column with missings'].fillna(mean value, inplace=True)
df['column with missings'].fillna(median value, inplace=True)

In [75]:
#before creating a new df with removed missing values 
df_prods.shape

(49693, 5)

In [76]:
#removed missing values and compare shape
df_prods_clean=df_prods[df_prods['product_name'].isnull()==False]
df_prods_clean.shape

(49677, 5)

Another way to drop missing values is through the following
df_prods.dropna(inplace = True)
df_prods.dropna(subset = [‘product_name’], inplace = True)
This overwrites the df instead of creating a new one
Default to the inplace argument is inplace=false which leaves the original df untouched and will return a view of the changed df

In [77]:
df_ords.isnull().sum()

Unnamed: 0                    0
order_id                      0
user_id                       0
order_number                  0
order_day_of_week             0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

There are 206209 missing values in the days_since_prior_order column. Missing values could indicate human or machine error or a missing value could mean that the user never ordered anything before. Because of this I will create a new flag column that includes the missing values. I will not replace the null values with anything becuase it would disrupt the analyzation. 

In [79]:
#flag column 
first_order_flag=df_ords[df_ords['days_since_last_order'].isnull()==False]

In [3]:
df_merged_large.isnull().sum()

Unnamed: 0.1                   0
Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
order_day_of_week              0
order_hour_of_day              0
days_since_last_order    2078068
product_id                     0
add_to_cart_order              0
reordered                      0
_merge                         0
dtype: int64

# Duplicates

In [80]:
df_prods_nodupes = df_prods_clean[df_prods_clean.duplicated()]

In [81]:
df_prods_nodupes

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [82]:
df_prods_clean_nodupes=df_prods_clean.drop_duplicates()
df_prods_clean_nodupes.shape

(49672, 5)

In [83]:
df_prods_clean.shape

(49677, 5)

In [84]:
df_ords_nodupes=df_ords.drop_duplicates()
df_ords.shape

(3421083, 7)

In [85]:
df_ords_nodupes.shape

(3421083, 7)

# 4.5 Questions

In [86]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


The decription for the order_id, user_id, and order_number are not relevant because they are used as identification values and not data values. The order DOW looks good. There are seven values including a value of 0 that I assume represents Sunday. order_hour_of_day also looks good with a min of 0 and a max of 23. The data in the days_since_prior_order alos looks good with a mni of 0 and a max of 30. 

In [87]:
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


The maximum value under the prices column is $99,999. I highly suspect that this is an error considering the context. 

There are no mix-type data in df_ords

There are no dupes in df_ords

Adressing the df_ords missing value by flagging 

In [88]:
#exporting 
df_prods_clean_nodupes.to_csv(os.path.join('/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data', 'products_cleaned.csv'))
df_ords_nodupes.to_csv(os.path.join('/Users/jessicagiron/Desktop/Instacart Basket Analysis 2023/02 Data/Prepared Data', 'orders_cleaned.csv'))